# Метод опорных векторов 
(Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Метод опорных векторов имеет еще одну особенность. Если преобразовать его оптимизационную задачу, то окажется, что итоговый классификатор можно представить как взвешенную сумму скалярных произведений данного объекта на объекты обучающей выборки:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/0QyGrriIEeWfQgrIyj-wEQ_c0c52535894b1bbdcb49917053062ae4_svmpic.png?expiry=1588118400000&amp;hmac=SOzjE0JH_9Tp1xe1tWH9Xd_QuQJI4u10GW1iS3VISi8" alt="" data-asset-id="0QyGrriIEeWfQgrIyj-wEQ">

По сути, алгоритм делает предсказания на основе сходства нового объекта с объектами обучающей выборки. При этом, как правило, далеко не все коэффициенты оказываются ненулевыми. Это означает, что классификация делается на основе сходства лишь с частью обучающих объектов. Такие объекты называются опорными.

Реализация в Scikit-Learn
Метод опорных векторов реализован в классе sklearn.svm.SVC.

Основными параметрами этого класса являются коэффициент С и тип ядра kernel. В данной задаче мы будем использовать линейное ядро — для этого нужно задать значение параметра kernel='linear'

Индексы опорных объектов обученного классификатора хранятся в поле support_

## Задача
1. Загрузите выборку из файла svm-data.csv. В нем записана двумерная выборка (целевая переменная указана в первом столбце, признаки — во втором и третьем).

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC

In [2]:
svm_data = pd.read_csv('_f6284c13db83a3074c2b987f714f24f5_svm-data.csv', 
                       header=None, 
                       names=['target', 'feature_1', 'feature_2'])
svm_data

,target,feature_1,feature_2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39
5,1.0,0.34,0.73
6,0.0,0.44,0.06
7,1.0,0.40,0.74
8,0.0,0.18,0.18
9,1.0,0.53,0.53


In [3]:
y_train = svm_data.pop('target')
y_train

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
5    1.0
6    0.0
7    1.0
8    0.0
9    1.0
Name: target, dtype: float64

2. Обучите классификатор с линейным ядром, параметром C = 100000 и random_state=241. Такое значение параметра нужно использовать, чтобы убедиться, что SVM работает с выборкой как с линейно разделимой. При более низких значениях параметра алгоритм будет настраиваться с учетом слагаемого в функционале, штрафующего за маленькие отступы, из-за чего результат может не совпасть с решением классической задачи SVM для линейно разделимой выборки.

In [4]:
clf_svm = SVC(C=100000, random_state=241, kernel='linear')

In [5]:
clf_svm.fit(svm_data, y_train)

SVC(C=100000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

3. Найдите номера объектов, которые являются опорными (нумерация с единицы). Они будут являться ответом на задание. Обратите внимание, что в качестве ответа нужно привести номера объектов в возрастающем порядке через запятую или пробел. Нумерация начинается с 1.

In [6]:
[i + 1 for i in sorted(clf_svm.support_)] # Опорными являются три объекта этой выборки.

[4, 5, 10]

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

## Данные
Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [7]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
newsgroups.target

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

Подбор параметров удобно делать с помощью класса sklearn.model_selection.GridSearchCV.

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

for a in gs.grid_scores_:
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров

2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
X

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [10]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
clf_svc = SVC(kernel='linear', random_state=241)
search = GridSearchCV(clf_svc, 
                      grid, 
                      scoring='accuracy', 
                      cv=KFold(n_splits=5, shuffle=True, random_state=241), 
                      n_jobs=-1)

In [11]:
search.fit(X, newsgroups.target)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=241, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [12]:
search.best_params_

{'C': 1.0}

4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [13]:
clf_svc = SVC(C=1, kernel='linear', random_state=241)

In [14]:
clf_svc.fit(X, newsgroups.target)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [15]:
clf_svc.coef_

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [47]:
coefs = abs(clf_svc.coef_.todense().A1)
coefs = np.argsort(coefs)

X = vectorizer.get_feature_names()
words = [X[i] for i in coefs[-10:]]

' '.join(sorted(words))

'atheism atheists bible god keith moon religion sci sky space'

In [45]:
# альтернативный способ:

word = pd.DataFrame(data=vectorizer.get_feature_names())
coef = pd.DataFrame(data=np.abs(np.asarray(clf_svc.coef_.todense()).reshape(-1)))
data = pd.concat([word, coef], axis=1)
data.columns = ['word', 'coef']
data.sort_values(by=['coef'])[-10:]

,word,coef
22936,sci,1.029307
15606,keith,1.097094
5776,bible,1.130612
21850,religion,1.139081
23673,sky,1.180132
17802,moon,1.201611
5093,atheists,1.249180
5088,atheism,1.254690
12871,god,1.920379
24019,space,2.663165
